In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import io
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')


KeyboardInterrupt



In [ ]:
data_path = '/content/drive/My Drive/Colab Notebooks/Tensor_decom_sample/data/'

In [ ]:
data=pd.read_csv(data_path + 'tripcsvbame.csv')

In [ ]:
data

In [ ]:
data['usage-stats/'][301:306]

In [ ]:
data_2023=data['usage-stats/'][301:306]
frames=[]
for x in data_2023:
    frames.append(pd.read_csv(data_path + x[1:]))

result = pd.concat(frames)
result.columns

In [ ]:
dict_columns = {
    "Number": "Rental Id",
    "Total duration (ms)": "Duration",
    "Bike number" : "Bike Id",
    "Start station" : 'StartStation Name',
    'End station': 'EndStation Name',
    'End Date': 'End date',
    'Start Date': 'Start date'
}

In [ ]:
result_general = result.rename(columns=dict_columns)
result_general['Duration'] = result_general['Duration']/1000
result_general

In [ ]:
date_format = '%d/%m/%Y %H:%M:%S'
result_general['End date']=pd.to_datetime(result_general['End date'])
result_general['Start date']=pd.to_datetime(result_general['Start date'])
result_general

In [ ]:
results_f = result_general[result_general['Duration']<3600]

In [ ]:
results_f

In [ ]:
len(result_general)

In [ ]:
len(results_f)

In [ ]:
result=results_f.dropna()
new_result=result.set_index('Start date').groupby([pd.Grouper(freq='h'), 'StartStation Name', 'EndStation Name']).count()
new_result

In [ ]:
new_result.columns

In [ ]:
new_result['No of trips']=new_result['Duration']

In [ ]:
new_result

In [ ]:
new_result = new_result[['No of trips']]

In [ ]:
new_result

In [ ]:
df = new_result.reset_index()
df = df.sort_values(by=['Start date'])
df = new_result.reset_index()
df

In [ ]:
total_stations = pd.concat([df['StartStation Name'],df['EndStation Name']])
total_uniq_stations = np.unique(total_stations).tolist()
len(total_uniq_stations)

In [ ]:
total_hours = np.unique(df['Start date']).tolist()
len(total_hours)

In [ ]:
i = len(total_uniq_stations)
k = len(total_hours)
trips_3darray = np.zeros((i,i,k))

In [ ]:
timeframe_pos = 0
for trip in df.index:
    so = total_uniq_stations.index(df['StartStation Name'][trip])
    sd = total_uniq_stations.index(df['EndStation Name'][trip])
    trips_3darray[so][sd][timeframe_pos] += 1
    if trip < df.index.stop-1:
        if df['Start date'][trip] != df['Start date'][trip+1]:
            timeframe_pos += 1

In [ ]:
trips_3darray

In [ ]:
trips_3darray[:,:,1].sum()

In [ ]:
trips_3darray.shape

In [ ]:
all_trips=trips_3darray[:,:,0]
all_trips

In [ ]:
plt.matshow(all_trips)
plt.colorbar()
plt.show()

## Tensor Decomposition

In [ ]:
!pip install tensorly

In [ ]:
import tensorly as tl
from tensorly.decomposition import tucker

In [ ]:
#Represent the trips as a tensor
tensor_trip = tl.tensor(trips_3darray, dtype='float64')

In [ ]:
tensor_trip.shape

In [ ]:
# Rank of Tucker decomposition
tucker_rank = [100, 100, 150]

In [ ]:
core, factors = tucker(tensor_trip, rank=tucker_rank, init='random', tol=10e-5,random_state=12345)

In [ ]:
#Convert the tucker tensor into a full tensor
tucker_rec = tl.tucker_to_tensor((core, factors))

In [ ]:
factors[0].shape

In [ ]:
factors[1].shape

In [ ]:
factors[2].shape

In [ ]:
import requests

In [ ]:
response= requests.get('https://api.tfl.gov.uk/BikePoint/')
stations = response.json()

In [ ]:
data =  pd.DataFrame()
s_names = []
s_lat = []
s_lon = []

NbBikes = []
NbEmptyDocks = []
NbDocks = []
NbStandardBikes = []
NbEBikes = []
s_id = []

In [ ]:
len_lat = 0
for station in stations:
    s_names.append(station['commonName'])
    s_lat.append(float(station['lat']))
    s_lon.append(float(station['lon']))
    for add_property in station['additionalProperties']:
        if add_property['key'] == 'NbBikes':
            NbBikes.append(int(add_property['value']))
        if add_property['key'] == 'NbEmptyDocks':
            NbEmptyDocks.append(int(add_property['value']))
        if add_property['key'] == 'NbDocks':
            NbDocks.append(int(add_property['value']))
        if add_property['key'] == 'NbStandardBikes':
            NbStandardBikes.append(int(add_property['value']))
        if add_property['key'] == 'NbEBikes':
            NbEBikes.append(int(add_property['value']))
        if add_property['key'] == 'TerminalName':
            s_id.append(int(add_property['value']))


In [ ]:
data['Station_name'] = s_names
data['s_lat'] = s_lat
data['s_lon'] = s_lon
data['s_num_bikes'] = NbBikes
data['s_num_empty_docks'] = NbEmptyDocks
data['s_num_std_bikes'] = NbStandardBikes
data['s_num_e-bikes'] = NbEBikes
data['s_num_docks'] = data['s_num_bikes'] + data['s_num_empty_docks']
data['s_id'] = s_id

In [ ]:
data

In [ ]:
# South East limits
se_limit_lat = 51.523811 # everthing lower than this value is south east
se_limit_lon = -0.101342 # everything greater than this value is south east

# East limits
e_limit_lat = 51.523811 # everything greater than this value is east
e_limit_lon = -0.022306 # everything greater than this is east

# South West limits
sw_limit_lat = 51.501631 # everthing lower than this value is south east
sw_limit_lon = -0.101342 # everything lower than this value is south east

# West limits
w_limit_lat = 51.501072  # everthing greater than this value is south east
w_limit_lon = -0.182406  # everthing lower than this value is south east

# Central limits
c_toplimit_lat = 51.501072 # everthing greater than this value is central
c_lowlimit_lat = 51.523811 # everything lower than this value is central

c_toplimit_lon = -0.101342 # everything lower than this value is central
c_lowlimit_lon = -0.182406 # everthing greater than this value is central

# North limits
n_toplimit_lon = -0.022306 # everything lower than this value is central
n_lowlimit_lon = -0.182406 # everthing greater than this value is central

n_lowlimit_lat = 51.523811 # everthing greater than this value is central

In [ ]:
loc = []

In [ ]:
for station in data.index:
    if data['s_lat'][station] > c_toplimit_lat and data['s_lat'][station]< c_lowlimit_lat and data['s_lon'][station]< c_toplimit_lon and data['s_lon'][station]>c_lowlimit_lon:
        loc.append(1) # Central
    elif data['s_lon'][station] < n_toplimit_lon and data['s_lon'][station] > n_lowlimit_lon and data['s_lat'][station] > n_lowlimit_lat:
        loc.append(2) # North
    elif data['s_lon'][station] < w_limit_lon and data['s_lat'][station] > w_limit_lat:
        loc.append(3) # West
    elif data['s_lon'][station] < sw_limit_lon and data['s_lat'][station] < sw_limit_lat:
        loc.append(4)  # South_West
    elif data['s_lon'][station] > e_limit_lon and data['s_lat'][station] > e_limit_lat:
        loc.append(5) # East
    elif data['s_lon'][station] > se_limit_lon and data['s_lat'][station] < se_limit_lat:
        loc.append(6) # South East
    else:
        loc.append('Other')

In [ ]:
data['Area_loc'] = loc

In [ ]:
data

In [ ]:
data.loc[data.Station_name == 'Chepstow Villas, Notting Hill'].values[0][9]

In [ ]:
color_stations = []
for station in total_uniq_stations:
  info_val = data.loc[data.Station_name == station].values
  if len(info_val)>0:
    color_stations.append(data.loc[data.Station_name == station].values[0][9])
  else:
    color_stations.append(7)

In [ ]:
len(color_stations)

In [ ]:
fig, axs = plt.subplots(5, 3)
fig.set_size_inches(18.5, 10.5)
axs[0, 0].scatter(x=range(len(factors[0])), y=np.transpose(factors[0])[0], c=color_stations)
axs[0, 1].scatter(x=range(len(factors[1])), y=np.transpose(factors[1])[0], c=color_stations)
axs[0, 2].plot(np.transpose(factors[2])[0])

axs[1, 0].scatter(x=range(len(factors[0])), y=np.transpose(factors[0])[1], c=color_stations)
axs[1, 1].scatter(x=range(len(factors[1])), y=np.transpose(factors[1])[1], c=color_stations)
axs[1, 2].plot(np.transpose(factors[2])[1])

axs[2, 0].scatter(x=range(len(factors[0])), y=np.transpose(factors[0])[2], c=color_stations)
axs[2, 1].scatter(x=range(len(factors[1])), y=np.transpose(factors[1])[2], c=color_stations)
axs[2, 2].plot(np.transpose(factors[2])[2])

axs[3, 0].scatter(x=range(len(factors[0])), y=np.transpose(factors[0])[3], c=color_stations)
axs[3, 1].scatter(x=range(len(factors[1])), y=np.transpose(factors[1])[3], c=color_stations)
axs[3, 2].plot(np.transpose(factors[2])[3])

axs[4, 0].scatter(x=range(len(factors[0])), y=np.transpose(factors[0])[4], c=color_stations)
axs[4, 1].scatter(x=range(len(factors[1])), y=np.transpose(factors[1])[4], c=color_stations)
axs[4, 2].plot(np.transpose(factors[2])[4])

In [ ]:
factors[2].shape

In [ ]:
type(factors[2])

In [ ]:
from google.colab import files

In [ ]:
# convert array into dataframe
result_df = pd.DataFrame(factors[2])

In [ ]:
result_df['timestamp'] = pd.to_datetime(total_hours)

In [ ]:
result_df.to_csv('filename.csv')
files.download('filename.csv')